In [ ]:
! pip install tensorflow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.8.2
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


In [ ]:
#Download the dataset 

# Split the training set into 60% and 40% to end up with 15,000 examples for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteN50FJ2/imdb_reviews-train.tfrecord*...…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteN50FJ2/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteN50FJ2/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
#split the data into train, validation set and test set 

print("Train set:",len(train_data))
print("Test set:",len(test_data))
print("Validation set:",len(validation_data))

Train set: 15000
Test set: 25000
Validation set: 10000


In [ ]:
#The dataset has senstences of reviews given by viewers on movies. It's consists of positive and negetive reviews 

train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch


<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

Let's print 10 labels 

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

There are various models available on the tensorflow hub. There are many other pre-trained text embeddings from TFHub that can be used.

> **google/nnlm-en-dim128/2**- trained with the same NNLM architecture on the same data as google/nnlm-en-dim50/2, but with a larger embedding dimension. Larger dimensional embeddings can improve on your task but it may take longer to train your model. google/nnlm-en-dim128-with-normalization/2 - the same as google/nnlm-en-dim128/2, but with additional text normalization such as removing punctuation. This can help if the text in your task contains additional characters or punctuation. google/universal-sentence-encoder/4 - a much larger model yielding 512 dimensional embeddings trained with a deep averaging network (DAN) encoder.

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape = [], dtype = tf.string, trainable = True)
hub_layer(train_examples_batch[:3])

#This creates a array of vectors for 3 senstences. Each word is given a number between -infinity to +infinity
#Note that no matter the length of the input text, the output shape() of the embeddings is: (num_examples, embedding_dimension).

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

building Model 

In [ ]:
#Building a sequential model with pre trained neutral network from the hub and activation layer as "ReLu" and one output layer

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


The layers are stacked sequentially to build the classifier:

1.   The first layer is a TensorFlow Hub layer.
2.   This layer uses a pre-trained Saved Model to map a sentence into its embedding vector. The pre-trained text embedding model that you are using (google/nnlm-en-dim50/2) splits the sentence into tokens, embeds each token and then combines the embedding. The resulting dimensions are: (num_examples, embedding_dimension). For this NNLM model, the embedding_dimension is 50.
This fixed-length output vector is piped through a fully-connected (Dense) layer with 16 hidden units.
3. The last layer is densely connected with a single output node.



 


Loss function and optimizer


> A model needs a loss function and an optimizer for training. Since this is a binary classification problem and the model outputs logits (a single-unit layer with a linear activation), you'll use the **binary_crossentropy** loss function.

> This isn't the only choice for a loss function, you could, for instance, choose mean_squared_error. But, generally, binary_crossentropy is better for dealing with probabilities—it measures the "distance" between probability distributions, or in our case, between the ground-truth distribution and the predictions.



> As optimizer **Adam** is used. There are other optimizers such as Adamax(implements the Adamax algorithm),SDG(Gradient descent (with momentum) optimizer),Optimizer(Base class for Keras optimizers) etc 





In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Train the model


> Train the model for 10 epochs in mini-batches of 512 samples. This is 10 iterations over all samples in the x_train and y_train tensors. While training, monitor the model's loss and accuracy on the 10,000 samples from the validation set:



In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 9s 254ms/step - loss: 0.6748 - accuracy: 0.5100 - val_loss: 0.6140 - val_accuracy: 0.5568
Epoch 2/10
30/30 [==============================] - 7s 218ms/step - loss: 0.5462 - accuracy: 0.6851 - val_loss: 0.4988 - val_accuracy: 0.7394
Epoch 3/10
30/30 [==============================] - 7s 222ms/step - loss: 0.4081 - accuracy: 0.8146 - val_loss: 0.4003 - val_accuracy: 0.8208
Epoch 4/10
30/30 [==============================] - 7s 220ms/step - loss: 0.2951 - accuracy: 0.8859 - val_loss: 0.3461 - val_accuracy: 0.8517
Epoch 5/10
30/30 [==============================] - 7s 221ms/step - loss: 0.2187 - accuracy: 0.9223 - val_loss: 0.3201 - val_accuracy: 0.8587
Epoch 6/10
30/30 [==============================] - 7s 222ms/step - loss: 0.1640 - accuracy: 0.9475 - val_loss: 0.3082 - val_accuracy: 0.8651
Epoch 7/10
30/30 [==============================] - 7s 225ms/step - loss: 0.1229 - accuracy: 0.9652 - val_loss: 0.3049 - val_accuracy: 0.8720
Epoch 

Evaulate the model



In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3528 - accuracy: 0.8533 - 2s/epoch - 48ms/step
loss: 0.353
accuracy: 0.853


Improvising to check if it can increase the accuarcy of the model and reduce the loss 


1. Add another Dense layer of 128 nodes
2. Increase the current number of nodes in the layer to 256


## **Remodelling**

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
# model.add(tf.keras.layers.Dense(16, activation='relu'))
# model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='softmax'))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense_18 (Dense)            (None, 128)               6528      
                                                                 
 dense_19 (Dense)            (None, 256)               33024     
                                                                 
 dense_20 (Dense)            (None, 1)                 257       
                                                                 
Total params: 48,230,409
Trainable params: 48,230,409
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adamax',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1) 

Epoch 1/10
30/30 [==============================] - 8s 245ms/step - loss: 7.6348 - accuracy: 0.4993 - val_loss: 7.6094 - val_accuracy: 0.5010
Epoch 2/10
30/30 [==============================] - 7s 233ms/step - loss: 7.6348 - accuracy: 0.4993 - val_loss: 7.6094 - val_accuracy: 0.5010
Epoch 3/10
30/30 [==============================] - 7s 237ms/step - loss: 7.6348 - accuracy: 0.4993 - val_loss: 7.6094 - val_accuracy: 0.5010
Epoch 4/10
30/30 [==============================] - 7s 236ms/step - loss: 7.6348 - accuracy: 0.4993 - val_loss: 7.6094 - val_accuracy: 0.5010
Epoch 5/10
30/30 [==============================] - 7s 231ms/step - loss: 7.6348 - accuracy: 0.4993 - val_loss: 7.6094 - val_accuracy: 0.5010
Epoch 6/10
30/30 [==============================] - 7s 230ms/step - loss: 7.6348 - accuracy: 0.4993 - val_loss: 7.6094 - val_accuracy: 0.5010
Epoch 7/10
30/30 [==============================] - 7s 231ms/step - loss: 7.6348 - accuracy: 0.4993 - val_loss: 7.6094 - val_accuracy: 0.5010
Epoch 

In [ ]:

results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 7.6240 - accuracy: 0.5000 - 2s/epoch - 37ms/step
loss: 7.624
accuracy: 0.500


Hypothesis : 

different optimizers, loss functions, dropout, regularizations, activation functions and observe the change in performance as you tune these hyper parameters.

Optimizers : 
adamax, Ftrl,Nadam, Optimizer, Optimizer,SGD



1. Binary Classification Loss Functions
* Binary Cross-Entropy
* Hinge Loss
* Squared Hinge Loss


2. Multi-Class Classification Loss Functions
* Multi-Class Cross-Entropy Loss
* Sparse Multiclass Cross-Entropy Loss
* Kullback Leibler Divergence Loss

regularization




1. Hidden layers : 16,32 ; acc : 0.850 ; loss : 0.562 ; Optimizer : Adam ; epochs = 10; batch = 512
2. Hidden layers : 16,128 ; acc : 0.849 ; loss : 0.656 ; Optimizer : Adam 
3. Hidden layers : 16,32,128 ; acc : 0.849 ; loss : 0.845 ; Optimizer : Adam
4. Hidden layers : 16,32,128 ; acc : 0.849 ; loss : 0.845 ; Optimizer : Adam


5. Hidden layers : 128,256 ; acc : 0.858 ; loss : 0.490 ;Activation function : reLu , softmax;  Optimizer : Adam ; loss : BinaryCrossentropy ; epochs : 20 
6. Hidden layers : 32,128,256 ; acc : 0.500 ; loss : 0.693 ;Activation function : reLu , softmax,relu;  Optimizer : SGD ; loss : BinaryCrossentropy ; epochs : 20
7. Hidden layers : 32,128,256 ; acc : 0.853 ; loss : 0.334 ;Activation function : reLu , softmax,relu; Optimizer : adamax ; loss : Hinge ; epochs : 20
8. Hidden layers : 32,128,256 ; acc : 0.846 ; loss : 0.583 ;Activation function : reLu , softmax,relu; Optimizer : adamax ; loss : SquaredHinge ; epochs : 20
9. Hidden layers : 256 ; acc : 0.500 ; loss : 7.624 ;Activation function : reLu ; Optimizer : SGD ; loss : BinaryCrossentropy ; epochs : 10 







